# Markov process

## Overview

While it is encouraging to see that we can predict daily bike arrivals based on departure data and conditional probabilities of arriving at each station, the plot thickens when we try to pinpoint the probabilities of a specific bike arriving at a given station. 

In our system, a bike ride is an indpendent event. If subscriber 101 decides to rent a bike at 9am downtown, that beahvior does not influence . Given that a bike trip in a given interval can be thought of as an independent, random event from a set of all possible bike trips during that interval, we can represent each time interval as a matrix of all possible bike trips in that time interval, and approach the problem using a Markov process. 

Here are three questions we want to answer: 

* Q1: If a bike leavs a station in our network at 9am, what is the probability it will arrive at Station 70 at 9:10am?

* Q2: If we know what station the bike left from, even better. Let's say it was Station 1. Given that the bike left Station 1 at 9am, what is the probability it will arrive at Station 70 at 9:10am?

* Q3: And finally, here's the kicker. Can knowing the destinations of all bikes that have left so far that strengthen our prediction? (weird question - Bayes rule - can we update our priors)

We pick a single observation moment in time, and construct a markov chain from that moment into the future. At this point we are only interested in knowing the arrival stations and times for all bikes that have left our stations up to the moment of the observation. We have sent the bikes out into the world - where and when will they land?

To build our chain we can construct a Markov matrix for each time interval. Given what we learned in our EDA, the majority of bike trips are less than 30 minutes, so we will start by calculating three 10 minute deltas (10, 20, 30). These deltas will indicate the probability of our bikes landing in each station between the observation point time and the observation point time plus the delta value. 

For example, our matrices for these three time deltas will look something like FIGURE 1 below, with each row representating the probability of a an arrival at a station given its departure from another station. For example, for delta_10, P(1|84) would be the probability of arriving at station 1 within ten minutes after the observation moemnt given a departure from station 84). 

In [13]:
# FIGURE 1

# delta_5  = [[P(1|1), P(1|2), P(1|3), ..., P(1|84)],
#             [P(2|1), P(2|2), P(2|3), ..., P(2|84)],
#             ...
#             [P(84|1), P(2|2), P(2|3), ..., P(84|84)]]

# delta_10 = [[P(1|1), P(1|2), P(1|3), ..., P(1|84)],
#             [P(2|1), P(2|2), P(2|3), ..., P(2|84)],
#             ...
#             [P(84|1), P(2|2), P(2|3), ..., P(84|84)]]

# delta_15 = [[P(1|1), P(1|2), P(1|3), ..., P(1|84)],
#             [P(2|1), P(2|2), P(2|3), ..., P(2|84)],
#             ...
#             [P(84|1), P(2|2), P(2|3), ..., P(84|84)]]


## Load data

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pprint
%load_ext autoreload
from pylab import rcParams
import matplotlib.patches as mpatches

In [21]:
may_2015 = pd.read_csv('~/Code/bikesupply/data/test_data/trips_may2015.csv', parse_dates = ['start_date'])
may_2016 = pd.read_csv('~/Code/bikesupply/data/test_data/trips_may2016.csv', parse_dates = ['start_date'])

## Time

In [22]:
def create_time_columns(data, date_column, hour_column, minutes_column, delta_column):
    data[date_column] = pd.to_datetime(data[date_column])
    data = data.set_index(date_column)
    data[hour_column] = [ts.hour for ts in data.index]
    data[minutes_column] = [ts.minute for ts in data.index]
    data[delta_column] = (data[hour_column] * 60) + data[minutes_column]
    data = data.reset_index()
    return data

In [23]:
may_2015 = create_time_columns(may_2015, 'start_date', 'start_hour', 'start_minutes', 'start_time_in_mins')  
may_2015 = create_time_columns(may_2015, 'end_date', 'end_hour', 'end_minutes', 'end_time_in_mins')

In [284]:
may_2016 = create_time_columns(may_2016, 'start_date', 'start_hour', 'start_minutes', 'start_time_in_mins')  
may_2016 = create_time_columns(may_2016, 'end_date', 'end_hour', 'end_minutes', 'end_time_in_mins')

In [234]:
# What is the historical probability distribution for that time of day
may_2015['weekday'] = may_2015.start_date.dt.weekday
may_2015_weekdays = may_2015.query("weekday >= 0 and weekday <= 4")

In [319]:
# Returns matrix of historical probabilities of station arrivals for specified time interval

def markov_matrix(data, observation_time, time_interval):
    d = {}
    for i in range(1, 85):
        magic = "start_station_id == {0} and start_time_in_mins < {1} and start_time_in_mins >= {1} - {2} and end_time_in_mins > {1}"
        stations = data.query(magic.format(i, observation_time, time_interval))
        if len(stations) > 0:
           d[i] = dict(stations.groupby(['end_station_id'])['trip_id'].count()/ len(stations))
           d[i] = {k:round(v, 2) for k, v in d[i].items()}
    return d

In [350]:
# markov_matrix(may_2015_weekdays, 540, 5) # 9am 

In [316]:
# Returns a dictionary of scaler values of bikes that have left each station in the specified time interval

def bikes_departed(start_day, data, observation_time, time_interval):
    d = {}
    for i in range(1, 85):
        magic = "start_station_id == {0} and start_day == {3} and start_time_in_mins < {1} and start_time_in_mins >= {1} - {2} and end_time_in_mins > {1}"
        stations = data.query(magic.format(i, observation_time, time_interval, start_day))
        if len(stations) > 0:
           d[i] = dict(stations.groupby(['end_station_id'])['trip_id'].count())
           d[i] = {k:round(v, 2) for k, v in d[i].items()}
    return d

In [352]:
# bikes_departed(2, may_2016, 540, 5) # Validation day May 2, 2016 (Monday at 9am)

### Cascading bins

The distribution of arrival times for bikes entered the system in the past 5 minutes will play out in a left-leaning distribution (most rides ending within the first 10 minutes). If we were to only count the bikes arriving from our current data subset, we would miss the longer-ride arrivals from previous time intervals. We will want to calculate our target estimates by taking a slice of the cascading matrix:

||Time Slice 1| Time Slice 3|TARGET BIN
|--------------|--------------- | --------------- | ---------------
|Interval 1|departures + 5|departures + 10|departures + 15
|Interval 2||departures + 5|departures + 10
|Interval_3 |||departures + 5



In [334]:
# Returns dictionary of markov matrices of historical probs for each five minute interval preceding the obs_time

def train_cascading_matrices(data, obs_time):
    d = {}
    for i in range(5, 30, 5):
        d[i] = markov_matrix(data, obs_time, i)
    return d

def test_cascading_matrices(start_day, data, obs_time):
    d = {}
    for i in range(5, 30, 5):
        d[i] = bikes_departed(start_day, data, obs_time, i)
    return d

In [359]:
train_cascading_matrices(may_2015_weekdays, 540)

{5: {2: {4: 0.5, 13: 0.5},
  4: {12: 1.0},
  22: {26: 1.0},
  25: {25: 1.0},
  31: {29: 1.0},
  34: {37: 0.75, 38: 0.25},
  39: {69: 0.67, 72: 0.06, 74: 0.06, 75: 0.06, 77: 0.17},
  41: {70: 1.0},
  42: {45: 0.33, 47: 0.33, 56: 0.33},
  45: {42: 0.25, 68: 0.5, 73: 0.25},
  46: {51: 1.0},
  47: {46: 1.0},
  49: {45: 0.2, 60: 0.2, 65: 0.2, 69: 0.2, 76: 0.2},
  50: {39: 0.12,
   47: 0.03,
   50: 0.02,
   55: 0.05,
   57: 0.27,
   60: 0.05,
   61: 0.12,
   62: 0.05,
   63: 0.02,
   64: 0.11,
   65: 0.02,
   70: 0.12,
   76: 0.02,
   77: 0.03},
  51: {65: 0.5, 70: 0.5},
  54: {50: 0.17, 51: 0.08, 61: 0.08, 65: 0.17, 69: 0.17, 70: 0.08, 75: 0.25},
  55: {41: 0.04,
   45: 0.04,
   48: 0.09,
   54: 0.04,
   57: 0.09,
   60: 0.13,
   63: 0.04,
   65: 0.09,
   66: 0.13,
   67: 0.09,
   76: 0.13,
   82: 0.09},
  56: {48: 0.1, 61: 0.4, 70: 0.4, 82: 0.1},
  57: {60: 0.5, 63: 0.5},
  58: {69: 1.0},
  59: {47: 0.1, 57: 0.1, 65: 0.5, 69: 0.3},
  60: {41: 0.43, 42: 0.14, 45: 0.14, 72: 0.14, 77: 0.14},


In [358]:
test_cascading_matrices(2, may_2016, 540)

{5: {35: {36: 1.0},
  39: {67: 1.0},
  50: {61: 1.0},
  57: {49: 1.0},
  61: {70: 1.0},
  66: {49: 1.0},
  69: {41: 1.0,
   42: 1.0,
   47: 1.0,
   50: 2.0,
   51: 2.0,
   55: 1.0,
   60: 1.0,
   62: 1.0,
   63: 1.0,
   67: 1.0},
  70: {55: 2.0, 63: 1.0, 65: 1.0, 75: 1.0},
  72: {65: 1.0},
  77: {64: 1.0}},
 10: {2: {84: 1.0},
  35: {36: 1.0},
  39: {67: 1.0, 69: 1.0},
  50: {61: 1.0},
  57: {49: 1.0},
  61: {70: 1.0},
  66: {49: 1.0},
  69: {41: 1.0,
   42: 1.0,
   47: 1.0,
   50: 2.0,
   51: 2.0,
   55: 1.0,
   60: 1.0,
   62: 1.0,
   63: 1.0,
   67: 1.0,
   72: 1.0},
  70: {55: 2.0, 63: 2.0, 65: 1.0, 75: 1.0},
  72: {65: 1.0},
  74: {60: 1.0},
  77: {64: 1.0}},
 15: {2: {84: 1.0},
  35: {36: 1.0},
  39: {67: 1.0, 69: 1.0},
  50: {61: 1.0},
  57: {49: 1.0},
  61: {70: 1.0},
  66: {49: 1.0},
  69: {41: 1.0,
   42: 1.0,
   47: 1.0,
   50: 2.0,
   51: 2.0,
   55: 1.0,
   60: 1.0,
   62: 1.0,
   63: 1.0,
   67: 1.0,
   72: 1.0},
  70: {55: 2.0, 63: 2.0, 65: 1.0, 75: 1.0},
  72: {65: 1.0}

In [ ]:
# Given the number of bikes that left each station, and a baseline threshold, can we predict the bins 

# e.g. binned_data = markov_matrix * departure_matrix
# e.g. predicted_arrival = binned_data[binned_data >= 0.3]

## References

http://nbviewer.jupyter.org/github/markdregan/Bayesian-Modelling-in-Python/blob/master/Section%204.%20Bayesian%20regression.ipynb

http://pymc-devs.github.io/pymc3/examples.html#